In [ ]:
import requests
import time
import hashlib
import json
import hmac
import base64
import codecs
from time import sleep
import pandas as pd
import urllib.parse

def query(method, request, payload):
    if query != '' and method == 'GET':
        url = "https://api.bittrex.com/v3/" + str(request) + str('?') + str(urllib.parse.urlencode(query))
    elif method == "DELETE":
        url = "https://api.bittrex.com/v3/" + str(request) + str('/') + str(query)
    else:
        url = "https://api.bittrex.com/v3/" + str(request)

    api_key = 'key'
    secret = 'secret'
    api_timestamp = str(int(time.time() * 1000))

    if method == "POST":
        payload = json.dumps(query)
    else:
        payload = ''

    contentHash = hashlib.sha512(payload.encode()).hexdigest()
    pre_sign = api_timestamp+url+method+contentHash
    signature = hmac.new(secret.encode(), pre_sign.encode(), hashlib.sha512).hexdigest()
    headers = {'Api-Key': api_key, 'Api-Timestamp': api_timestamp, 'Api-Content-Hash': contentHash, 'Api-Signature': signature, 'Content-Type': 'application/json', 'Accept': 'application/json'}

    if method == "POST":
        try:
            r = requests.post(url = url, data=json.dumps(query), headers = headers)
        except error as e:
            print('Unable to POST')
    elif method == "DELETE":
        try:
            r = requests.delete(url = url, headers = headers)
        except error as e:
            print('Unable to DELETE')
    else:
        try:
            r = requests.get(url = url, params = payload, headers = headers)
        except error as e:
            print('Unable to GET')

    data = r.json()
    print(f'Order details:\n{data}')
    return data

In [ ]:
while True:
    try:
        btc_old = requests.get('https://api.bittrex.com/v3/markets/BTC-USD/ticker').json()
    except error as e:
        print('Unable to obtain ticker data')
    
    sleep(300)
    
    try:
        btc_new = requests.get('https://api.bittrex.com/v3/markets/BTC-USD/ticker').json()
    except error as e:
        print('Unable to obtain ticker data')
    
    percent = (((float(btc_new['bidRate']) - float(btc_old['bidRate'])) * 100) / float(btc_old['bidRate']))
    
    if percent >= 5:

        payload = {
            'marketSymbol': 'ETH-USD',
            'direction': 'BUY',
            'type': 'MARKET',
            'quantity': 0.07,
            'timeInForce': 'GOOD_TIL_CANCELLED',
            }
        
        try:
            order_id = (query(method='POST', request='orders', payload=payload))['id']
        except error as e:
            print('Unable to place Order')
        
        sleep(2)
        
        try:
            check_order = (query(method='GET', request='orders', payload=order_id))['status']
        except error as e:
            print('Unable to check order')
        
        if check_order == 'OPEN':
            print ('Order placed at {}'.format(pd.Timestamp.now()))
            break
        else:
            print ('Order canceled at {}'.format(pd.Timestamp.now()))
            break
    else:
        print(f'The trade requirement was not satisfied. The percentage move is at {percent}')
        continue